### Log into ArcGIS Online

In [15]:
from arcgis.gis import GIS

import warnings
from urllib3.exceptions import InsecureRequestWarning
warnings.simplefilter("ignore", InsecureRequestWarning)

gis = GIS("home")

In [16]:
cm_id = "03209b91e33f4c588007319d18cd6e70"
cm_item = gis.content.get(cm_id)

inventory_table = cm_item.tables[1]
dependencies_table = cm_item.tables[2]

In [ ]:
def get_status(status):
    if not status:
        return "None"
    else: return status.lstrip("org_").title()

def get_item_url(item):
    if item.type in ["Vector Tile Service", "Feature Service", "Map Service", "Image Service"]:
        return f'<a target="_blank" href="{item.url}">View Service</a>'
    elif item.type == "Web Map":
        return f'<a target="_blank" href="https://www.maps.arcgis.com/apps/mapviewer/index.html?webmap={item.id}">View Map</a>'
    elif item.type in ["Hub Site Application", "StoryMap", "Web Experience", "Web Experience Template", "Dashboard", "Hub Page"]:
        return f'<a target="_blank" href="{item.url}">View App</a>'
    else: return "Not applicable"

def get_item_page_url(item_id):
    return f'<a target="_blank" href="https://www.maps.arcgis.com/home/item.html?id={item_id}">View Item Page</a>'

In [ ]:
all_items = gis.content.search(query="*", max_items=10000)

items_list = []
dependencies_list = []

for item in all_items:

    items_dict = {}
    update_items_dict = {}
    
    items_dict["item_id"] = item.id
    items_dict["item_url"] = get_item_page_url(item.id)
    items_dict["service_url"] = get_item_url(item)
    items_dict["item_type"] = item.type
    items_dict["title"] = item.title
    items_dict["owner"] = item.owner
    items_dict["sharing"] = str(item.sharing.sharing_level.value).title()
    items_dict["status"] = get_status(item.content_status)
    items_dict["date_created"] = item.created
    items_dict["date_modified"] = item.modified

    # BEGIN DEPENDENCIES
    dependencies = item.get_dependencies(deep=True)

    if not dependencies:
        items_dict["has_dependencies"] = "No"
        items_dict["dependencies_count"] = 0

    else:
        items_dict["has_dependencies"] = "Yes"
        items_dict["dependencies_count"] = len(dependencies)

        for dependency in dependencies:

            dependencies_dict = {}
            update_dependencies_dict = {}

            dependencies_dict["origin_id"] = item.id
            dependencies_dict["origin_title"] = item.title
            dependencies_dict["dependent_id"] = dependency.id
            dependencies_dict["dependent_title"] = dependency.title
            dependencies_dict["dependent_type"] = dependency.type
            dependencies_dict["dependent_item_url"] = get_item_page_url(dependency.id)
            dependencies_dict["dependent_url"] = get_item_url(dependency)

            update_dependencies_dict["attributes"] = dependencies_dict
            dependencies_list.append(update_dependencies_dict)

    update_items_dict["attributes"] = items_dict
    items_list.append(update_items_dict)

print(f"Number of items to add to items table: {len(items_list)}")
print(f"Number of items to add to dependencies table: {len(dependencies_list)}")

Number of items to add to table: 223
Number of items to add to table: 402


### Add dependencies to the dependencies table

In [ ]:
# This script will effectively overwrite the output of the last run, 
# so before writing any new records we simply truncate tables
# NOTE: can't use truncate like for users table because of the relationship class!
# NOTE: Do we even need the relationship class? We're just shoving everything in a Dashboard/ExB anyway...?
inventory_table.delete_features(where="1=1")
items_update = inventory_table.edit_features(adds=items_list)

# Add items and dependencies to tables
dependencies_table.delete_features(where="1=1")
dependencies_update = dependencies_table.edit_features(adds=dependencies_list)